In [ ]:
import gurobipy as gb
import pandas as pd

Direct Flights Only Model

Question 2 - Data preparation for the direct flights model

In [ ]:
data = pd.read_csv("flights-1.csv",  encoding='latin-1')

# Fix of the issue with different decodings.
encoding_issues = {"ZÃ¼rich": "Zurich",
                   "DÃ¼sseldorf": "Dusseldorf", "MÃ¡laga": "Malaga"}
for key, value in encoding_issues.items():
    data.loc[data["departureCity"] == key, "departureCity"] = value
    data.loc[data["arrivalCity"] == key, "arrivalCity"] = value

# Create necessary dictionaries.
capacity = {"small": 50, "medium": 100, "large": 300}
cost_per_mile = {"small": 4.5, "medium": 8, "large": 20}
charge = 0.1

distances = data.set_index(['departureCity', 'arrivalCity'])[
    'Distance'].to_dict()
demands = data.set_index(['departureCity', 'arrivalCity'])['Demand'].to_dict()

route_list = list(distances.keys())

Question 3 - Gurobi model for direct flights framework

In [ ]:
model_Q1 = gb.Model()

small = model_Q1.addVars(distances, name="small", vtype=gb.GRB.INTEGER, lb=0)
medium = model_Q1.addVars(distances, name="medium", vtype=gb.GRB.INTEGER, lb=0)
large = model_Q1.addVars(distances, name="large", vtype=gb.GRB.INTEGER, lb=0)
num_of_passengers = model_Q1.addVars(
    distances, name="Num_of_pass_", vtype=gb.GRB.INTEGER, lb=0)

# Profit it the Objective of this model.
model_Q1.setObjective(gb.quicksum(charge*distances[route]*num_of_passengers[route] - distances[route]*(cost_per_mile["small"]*small[route] + cost_per_mile["medium"] * medium[route] + cost_per_mile["large"] * large[route])
                                  for route in route_list), gb.GRB.MAXIMIZE)

# The number of customers per route should be lower or equal to the demand.
model_Q1.addConstrs(num_of_passengers[route] <= demands[route]
                    for route in route_list)

# Number of customers per route should be equal or lower than each plane's full capacity by number of those planes.
model_Q1.addConstrs(num_of_passengers[route] <= capacity["small"] * small[route] + capacity["medium"] * medium[route] + capacity["large"] * large[route]
                    for route in route_list)

model_Q1.optimize()
model_Q1.ObjVal
all_vars = model_Q1.getVars()

# For the further analysis it was decided to move the variables and their values into a data frame format.
gurobi_variables = model_Q1.getAttr("X", all_vars)
names = model_Q1.getAttr("VarName", all_vars)

gurobi_result = pd.DataFrame({"names": names, "values": gurobi_variables})

Question 4 - All necessary KPIs for direct flights model

In [ ]:
# (a)
daily_profit = model_Q1.ObjVal

# (b)
small_result = gurobi_result[gurobi_result["names"].str.startswith(
    "small")]["values"].sum()
medium_result = gurobi_result[gurobi_result["names"].str.startswith(
    "medium")]["values"].sum()
large_result = gurobi_result[gurobi_result["names"].str.startswith(
    "large")]["values"].sum()

# (c)
# It was decided to create a united data frame with routes, distances, demand,
# Num of each plane type, and total amount of passengers. It is needed since we can't really
# see the revenue, costs and other proportions directly.


distances_df = pd.DataFrame(list(distances.items()), columns=["Route", "dist"])
demand_df = pd.DataFrame(list(demands.items()), columns=["Route", "demand"])
distance_and_demand_df = distances_df.merge(
    demand_df, on=["Route"], how="left")

# This for loop reshapes the data from long to wide.
# I create columns with the types of planes and num of passengers.
gurobi_variables = ["small", "medium", "large", "Num_of_pass_"]
gurobi_output_wide = pd.DataFrame()

for variable in gurobi_variables:
    resulting_df = gurobi_result[gurobi_result["names"].str.startswith(
        variable)]

    # The next rows just make sure that the Route columns are the same - Series of tuples.
    resulting_df = resulting_df.rename(
        columns={"values": f"{variable}_values"})
    resulting_df["Route"] = '(' + \
        resulting_df['names'].str.extract(rf'{variable}\[(.*?)\]') + ')'
    resulting_df.drop(columns=['names'], inplace=True)
    if gurobi_output_wide.empty:
        gurobi_output_wide = resulting_df
    else:
        gurobi_output_wide = gurobi_output_wide.merge(
            resulting_df, on=["Route"], how="left")

gurobi_output_wide["Route"] = gurobi_output_wide["Route"].str.strip(
    '()').str.split(',').apply(tuple)

# We merged them.
demand_distance_and_gurobi_wide = distance_and_demand_df.merge(
    gurobi_output_wide, on=["Route"], how="left")

revenue_raw = demand_distance_and_gurobi_wide["dist"] * \
    demand_distance_and_gurobi_wide["Num_of_pass__values"]*0.1
revenue = revenue_raw.sum()

costs_raw = demand_distance_and_gurobi_wide["dist"]*(4.5 * demand_distance_and_gurobi_wide["small_values"] + 8 *
                                                     demand_distance_and_gurobi_wide["medium_values"] + 20 * demand_distance_and_gurobi_wide["large_values"])
costs = costs_raw.sum()

# (d)
profit_margin = daily_profit/revenue

# (e)

daily_number_of_passengers = demand_distance_and_gurobi_wide["Num_of_pass__values"].sum(
)

# (f)
full_capacity = (50 * demand_distance_and_gurobi_wide["small_values"] + 100 *
                 demand_distance_and_gurobi_wide["medium_values"] +
                 300 * demand_distance_and_gurobi_wide["large_values"]).sum()
utilization = 1 - \
    (full_capacity -
     demand_distance_and_gurobi_wide["Num_of_pass__values"].sum()) / full_capacity

# (g)

# WE calculate what is the share of missed demand w.r.t the whole demand
lost_demand = (demand_distance_and_gurobi_wide["demand"].sum(
) - demand_distance_and_gurobi_wide["Num_of_pass__values"].sum())/demand_distance_and_gurobi_wide["demand"].sum()


Introducing a Hub and Spoke System

Question 6 - Data preparation for the one hub model 

In [ ]:
df = pd.read_csv("flights-1.csv",  encoding='latin-1')

issues = {"ZÃ¼rich": "Zurich",
          "DÃ¼sseldorf": "Dusseldorf", "MÃ¡laga": "Malaga"}
for key, value in issues.items():
    df.loc[df["departureCity"] == key, "departureCity"] = value
    df.loc[df["arrivalCity"] == key, "arrivalCity"] = value

df2 = df.set_index('departureCity').join(df.set_index('departureCity'),
                                         lsuffix = "first", rsuffix = "second").reset_index()
df2 = df2[df2.arrivalCityfirst != df2.arrivalCitysecond]
df2["distance"] = df2.Distancefirst + df2.Distancesecond
df2 = df2[["departureCity", "arrivalCityfirst", "arrivalCitysecond", "distance"]]
df2.columns = ["Hub", "arrivalCity", "departureCity", "Distance"]
# Plane size types
sizes = ["small", "medium", "large"]

# Cost per mile per plane type
cost_per_mile = {"small": 4.5 , "medium":8, "large": 20}

# Capacity per plane type
capacity = {"small":50 , "medium": 100 , "large": 300}

# Direct distance between city i and city j
direct_distance = {}
for index, row in df.iterrows():
    direct_distance[row.departureCity, row.arrivalCity] = row.Distance

# Demand for route between city i and city j
demand = {}
for index, row in df.iterrows():
    demand[row.departureCity, row.arrivalCity] = row.Demand

# Routes between city i and city j
direct_routes = []
for index, row in df.iterrows():
    direct_routes.append((row['departureCity'], row['arrivalCity']))


# Routes between city i, hub k, and city j
indirect_routes = []
for index, row in df2.iterrows():
    indirect_routes.append((row['departureCity'], row['Hub'], row['arrivalCity']))

# List of all cities that can be possible hubs
hub_list = df2['Hub'].unique().tolist()

# Revenue for the direct and indirect customer based on direct charge of 0.1 and indirect charge of 0.08
revenue_direct_customer = {}
for index, row in df.iterrows():
    revenue_direct_customer[row.departureCity, row.arrivalCity] = row.Distance * 0.1
    
revenue_indirect_customer = {}
for index, row in df2.iterrows():
    revenue_indirect_customer[row.departureCity, row.Hub ,row.arrivalCity] = row.Distance * 0.08

Question 7 - Gurobi model for one hub framework

In [ ]:
# Initialize an empty model
model_Q2 = gb.Model()

# Setting the runtime limit to 15 minutes
model_Q2.setParam('TimeLimit', 15*60)

# Telling the model to maximize the objective function
model_Q2.ModelSense = gb.GRB.MAXIMIZE

# Creating a variable for the different plane sizes
plane = {}
for size in sizes:
    plane[size] = model_Q2.addVars(direct_routes, vtype = gb.GRB.INTEGER, lb = 0, name = "{plane_type}")
    
# Direct passengers (x) and indirect passengers (f)
x = model_Q2.addVars(direct_routes, vtype = gb.GRB.INTEGER, lb = 0, name = "num_direct")
f = model_Q2.addVars(indirect_routes, vtype = gb.GRB.INTEGER, lb = 0, name = "num_indirect")

# Hub
hub = model_Q2.addVars(hub_list, vtype = gb.GRB.BINARY, name = "hub")

# Objective funtion to maximize profits
model_Q2.setObjective(gb.quicksum((x[ij])*revenue_direct_customer[ij]
                               for ij in direct_routes)
                    + gb.quicksum((f[ikj])*revenue_indirect_customer[ikj]
                                for ikj in indirect_routes)
                    - gb.quicksum((cost_per_mile[size] * plane[size][ij]*direct_distance[ij])
                                for size in sizes
                                for ij in direct_routes))

# The sum of direct and indirect passengers for all routes must be less than the capacity of the plane
model_Q2.addConstrs(x[ij] 
                  + gb.quicksum(f[ijk]
                                for ijk in indirect_routes
                                if ijk[0] == ij[0] and ijk[1] == ij[1])
                  + gb.quicksum(f[kij]
                                for kij in indirect_routes
                                if kij[1] == ij[0] and kij[2] == ij[1])
                  <= gb.quicksum(capacity[size]*plane[size][ij]
                                 for size in sizes)
                  for ij in direct_routes)

# The sum of direct and indirect passengers for all routes must be less than demand for that route
model_Q2.addConstrs(x[ij] +
                  gb.quicksum(f[ikj]
                              for ikj in indirect_routes
                              if ikj[0] == ij[0] and ikj[2] == ij[1])
                  <= demand[ij]
                  for ij in direct_routes)

# Large M value
M = 10000000000

# If there is a hub, there are indirect passengers, if no hub, no indirect passengers
model_Q2.addConstrs(f[ikj] <= M * hub[ikj[1]]
                  for ikj in indirect_routes)

# Total number of chosen hubs is one
model_Q2.addConstr(gb.quicksum(hub[k] for k in hub_list) == 1)

# Optimizing the model
model_Q2.optimize()

if not model_Q2.status == gb.GRB.OPTIMAL:
    print("Optimal Value:", model_Q2.ObjVal)

Question 8 - KPIs for the one hub model

In [ ]:
# Extract and print the optimal solution
print("Total Daily Profit:", model_Q2.ObjVal)

# Calculate revenue
total_direct_revenue = sum(x[ij].x * revenue_direct_customer[ij] for ij in direct_routes)
total_indirect_revenue = sum(f[ikj].x * revenue_indirect_customer[ikj] for ikj in indirect_routes)

total_revenue = total_direct_revenue + total_indirect_revenue

total_small_planes = sum(plane[" small "][ij].x for ij in direct_routes)
total_medium_planes = sum(plane[" medium "][ij].x for ij in direct_routes)
total_large_planes = sum(plane[" large "][ij].x for ij in direct_routes)

print("Total Small Planes:", total_small_planes)
print("Total Medium Planes:", total_medium_planes)
print("Total Large Planes:", total_large_planes)

print("Total Daily Revenue:", total_revenue)

total_cost = total_revenue - model_Q2.ObjVal

print("Total Daily Cost:", total_cost)

profit_margin = model_Q2.ObjVal / total_revenue

print("Profit Margin:", profit_margin)

total_passengers = sum(x[ij].x for ij in direct_routes) + sum(f[ikj].x for ikj in indirect_routes)

print("Total Daily Passengers:", total_passengers)

total_utlization = total_passengers / (total_small_planes * capacity[" small "] + total_medium_planes * capacity[" medium "] + total_large_planes * capacity[" large "])

print("Total Utilization:", total_utlization)

lost_demand_percentage = 1 - total_passengers / sum(demand[ij] for ij in direct_routes)

print("Lost Demand Percentage:", lost_demand_percentage)


selected_hubs = [hub_location for hub_location, is_selected in hub.items() if is_selected.x > 0.5]
print(f"Selected Hub(s): {', '.join(selected_hubs)}")

Question 9 - Gurobi model for the for the second hub framework

In [ ]:
# Initialize an empty model for 2 hubs
model_Q2hubs = gb.Model()

# Setting the runtime limit to 15 minutes
model_Q2hubs.setParam('TimeLimit', 15*60)

# Telling the model to maximize the objective function
model_Q2hubs.ModelSense = gb.GRB.MAXIMIZE

# Creating a variable for the different plane sizes
plane = {}
for size in sizes:
    plane[size] = model_Q2hubs.addVars(direct_routes, vtype = gb.GRB.INTEGER, lb = 0, name = "{plane_type}")
    
# Direct passengers (x) and indirect passengers (f)
x = model_Q2hubs.addVars(direct_routes, vtype = gb.GRB.INTEGER, lb = 0, name = "num_direct")
f = model_Q2hubs.addVars(indirect_routes, vtype = gb.GRB.INTEGER, lb = 0, name = "num_indirect")

# Hub
hub = model_Q2hubs.addVars(hub_list, vtype = gb.GRB.BINARY, name = "hub")

# Objective funtion to maximize profits
model_Q2hubs.setObjective(gb.quicksum((x[ij])*revenue_direct_customer[ij]
                               for ij in direct_routes)
                    + gb.quicksum((f[ikj])*revenue_indirect_customer[ikj]
                                for ikj in indirect_routes)
                    - gb.quicksum((cost_per_mile[size] * plane[size][ij]*direct_distance[ij])
                                for size in sizes
                                for ij in direct_routes))

# The sum of direct and indirect passengers for all routes must be less than the capacity of the plane
model_Q2hubs.addConstrs(x[ij] 
                  + gb.quicksum(f[ijk]
                                for ijk in indirect_routes
                                if ijk[0] == ij[0] and ijk[1] == ij[1])
                  + gb.quicksum(f[kij]
                                for kij in indirect_routes
                                if kij[1] == ij[0] and kij[2] == ij[1])
                  <= gb.quicksum(capacity[size]*plane[size][ij]
                                 for size in sizes)
                  for ij in direct_routes)

# The sum of direct and indirect passengers for all routes must be less than demand for that route
model_Q2hubs.addConstrs(x[ij] +
                  gb.quicksum(f[ikj]
                              for ikj in indirect_routes
                              if ikj[0] == ij[0] and ikj[2] == ij[1])
                  <= demand[ij]
                  for ij in direct_routes)

# Large M value
M = 10000000000

# If there is a hub, there are indirect passengers, if no hub, no indirect passengers
model_Q2hubs.addConstrs(f[ikj] <= M * hub[ikj[1]]
                  for ikj in indirect_routes)

# Total number of chosen hubs is one
model_Q2hubs.addConstr(gb.quicksum(hub[k] for k in hub_list) == 1)

# Optimizing the model
model_Q2hubs.optimize()

if not model_Q2hubs.status == gb.GRB.OPTIMAL:
    print("Optimal Value:", model_Q2hubs.ObjVal)